# Use Insurance Code Template Notebook Template

## Packages needed to run this notebook if running with inbuilt snowpark 3.8 template
##### ! pip install "snowflake-connector-python[pandas]"
##### ! sudo pip install snowflake-ml-python==1.0.11 -U
##### ! pip install --upgrade snowflake-snowpark-python==1.9.0
##### ! pip install --upgrade xgboost==1.7.3
##### ! pip install --upgrade numpy==1.24.3
##### ! pip install --upgrade pandas==1.5.3
##### ! pip install --upgrade anyio==3.5.0
##### ! pip install --upgrade packaging==23.1
##### ! pip install --upgrade scikit-learn==1.3.0
##### ! pip install --upgrade typing-extensions==4.7.1
##### ! pip install --upgrade cryptography==39.0.0
##### ! pip install --upgrade fsspec==2023.9.2
##### ! pip install fosforio
##### ! pip install fosforml

In [79]:
from snowflake.snowpark import Session
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
# import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
# Pandas Tools
from snowflake.connector.pandas_tools import write_pandas
# Data Science Libs
import numpy as np
import pandas as pd
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
# FosforIO to read from snowflake
from fosforio import snowflake
# FosforML to register Model on FDC
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

# Read data using Snowflake's Snowpark

In [80]:
#Import all snowflake connection details from Template or Project variables.
db_user = os.getenv('Snowflake_user')
db_password =  os.getenv('Snowflake_password')
db_account = os.getenv('Snowflake_Account')
db_database =  os.getenv('Snowflake_Database')
db_role = os.getenv('Snowflake_user')
db_warehouse = os.getenv('Snowflake_Warehouse')
db_schema = os.getenv('Snowflake_Schema')

In [81]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [82]:
session.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session.sql('use database FDC_Insurance;').collect()
session.sql('use schema FDC_Insurance.PUBLIC;').collect()

df = session.table('FDC_Insurance.PUBLIC.AUTO_INSURANCE_CLAIMS_DATA_PRODUCT')

In [83]:
df_backup = df.to_pandas().copy()

# 1. Claim Amount XGB Snowflake Model Prediction

In [84]:
from joblib import dump, load
filename = "Claims_Snowpark_XGB_Regression.joblib" #Model version 1

In [85]:
# Load the Snowflake's Snowpark XGB model
model = load('model_artifacts/' + filename)

In [86]:
df = df.to_pandas().replace(np.nan, pd.isna)

In [87]:
df["AUTO_YEAR"] = df["AUTO_YEAR"].astype(str)

In [88]:
pred = model.predict(df)
df_backup["CLAIM_AMOUNT_PREDICTION"] = pred["PREDICTION"]

# Merge XGB Model output in a back up dataframe for reference

In [89]:
df_backup.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,CLAIM_AMOUNT_PREDICTION
0,170,60,429383,05-05-2013,CT,500/1000,537,697,0,466132,...,No Police Report Available,101257.0,21316.0,6844.0,73097.0,Dodge,Neon,2013,No Fraud Reported,83311.593750
1,50,28,460493,18-04-2013,CT,500/1000,1727,1918,2919469,463809,...,Police Report Available,80444.0,472.0,1165.0,78807.0,Accura,TL,2005,No Fraud Reported,70663.523438
2,31,28,448474,31-07-2004,CT,250/500,952,1204,7957599,610393,...,Police Report Available,75641.0,7510.0,10412.0,57719.0,BMW,X5,2009,No Fraud Reported,71273.593750
3,188,58,462247,27-12-2010,CT,100/300,1521,1728,8639634,606942,...,No Police Report Available,80621.0,10134.0,908.0,69579.0,BMW,X5,2015,No Fraud Reported,75392.898438
4,24,65,436220,13-01-2003,CT,500/1000,1090,1535,3551771,431277,...,Unknown,59144.0,305.0,15598.0,43241.0,Nissan,Pathfinder,2014,Fraud Reported,69774.695312


# 2. Fraud Claim Decision Tree Classifier

In [90]:
import pickle

In [91]:
with open('model_artifacts/actual_model_columns.pkl', 'rb') as f:
    actual_model_columns = pickle.load(f) # deserialize using load()

In [92]:
with open('model_artifacts/cat_col.pkl', 'rb') as f:
    cat_col = pickle.load(f) # deserialize using load()

In [93]:
with open('model_artifacts/Decisiontree_Fraudclassifier_v1.pkl', 'rb') as f:
    dtc = pickle.load(f) # deserialize using load()

In [94]:
with open('model_artifacts/num_col.pkl', 'rb') as f:
    num_col = pickle.load(f) # deserialize using load()

In [95]:
with open('model_artifacts/num_col_test.pkl', 'rb') as f:
    num_col_test = pickle.load(f) # deserialize using load()

In [96]:
with open('model_artifacts/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f) # deserialize using load()

In [97]:
with open('model_artifacts/to_drop.pkl', 'rb') as f:
    to_drop = pickle.load(f) # deserialize using load()

In [98]:
df = df_backup.copy()

In [99]:
df.columns

Index(['MONTHS_AS_CUSTOMER', 'CUSTOMER_AGE', 'POLICY_NUMBER',
       'POLICY_BIND_DATE', 'POLICY_STATE', 'POLICY_CSL', 'POLICY_DEDUCTABLE',
       'POLICY_ANNUAL_PREMIUM', 'UMBRELLA_LIMIT', 'INSURED_ZIP', 'INSURED_SEX',
       'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION', 'INSURED_HOBBIES',
       'INSURED_RELATIONSHIP', 'CAPITAL_GAINS', 'CAPITAL_LOSS',
       'INCIDENT_DATE', 'INCIDENT_TYPE', 'COLLISION_TYPE', 'INCIDENT_SEVERITY',
       'AUTHORITIES_CONTACTED', 'INCIDENT_STATE', 'INCIDENT_CITY',
       'INCIDENT_LOCATION', 'INCIDENT_HOUR_OF_THE_DAY', 'INCIDENT_TIME_OF_DAY',
       'NUMBER_OF_VEHICLES_INVOLVED', 'PROPERTY_DAMAGE', 'BODILY_INJURIES',
       'WITNESSES', 'POLICE_REPORT_AVAILABLE', 'TOTAL_CLAIM_AMOUNT_PAID',
       'INJURY_CLAIM', 'PROPERTY_CLAIM', 'VEHICLE_CLAIM', 'AUTO_MAKE',
       'AUTO_MODEL', 'AUTO_YEAR', 'FRAUD_REPORTED', 'CLAIM_AMOUNT_PREDICTION'],
      dtype='object')

In [100]:
df.shape

(234265, 41)

In [101]:
df = df.fillna(0)
df.columns = df.columns.str.lower()
df.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)
df.drop(to_drop, inplace=True, axis=1)

In [102]:
# extracting categorical columns
cat_df = df[['customer_age','policy_csl','insured_sex','insured_education_level','insured_occupation','insured_relationship','incident_type','collision_type','incident_severity','authorities_contacted','incident_time_of_day','property_damage','police_report_available']]
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [103]:
num_df = df[['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim']]
    
X = pd.concat([num_df, cat_df], axis = 1)

In [104]:
num_df_test = X[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

scaled_data_test = scaler.transform(num_df_test)
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X.index)

In [105]:
X.drop(columns = scaled_num_df_test.columns, inplace = True)
X = pd.concat([scaled_num_df_test, X], axis = 1)

In [106]:
missing_features = [missing_col for missing_col in actual_model_columns if missing_col not in X.columns]
X[missing_features] = 0

In [107]:
prediction = dtc.predict(X[actual_model_columns])

In [108]:
probability = dtc.predict_proba(X)[:,1]

In [109]:
prediction

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [110]:
probability

array([1., 1., 1., ..., 1., 1., 1.])

In [111]:
df_backup['FRAUD_PROBABILITY'] = probability
df_backup['FRAUD_CLAIM_CLASSIFIER'] = prediction

In [112]:
df_backup.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,CLAIM_AMOUNT_PREDICTION,FRAUD_PROBABILITY,FRAUD_CLAIM_CLASSIFIER
0,170,60,429383,05-05-2013,CT,500/1000,537,697,0,466132,...,21316.0,6844.0,73097.0,Dodge,Neon,2013,No Fraud Reported,83311.593750,1.0,No Fraud Reported
1,50,28,460493,18-04-2013,CT,500/1000,1727,1918,2919469,463809,...,472.0,1165.0,78807.0,Accura,TL,2005,No Fraud Reported,70663.523438,1.0,No Fraud Reported
2,31,28,448474,31-07-2004,CT,250/500,952,1204,7957599,610393,...,7510.0,10412.0,57719.0,BMW,X5,2009,No Fraud Reported,71273.593750,1.0,No Fraud Reported
3,188,58,462247,27-12-2010,CT,100/300,1521,1728,8639634,606942,...,10134.0,908.0,69579.0,BMW,X5,2015,No Fraud Reported,75392.898438,1.0,No Fraud Reported
4,24,65,436220,13-01-2003,CT,500/1000,1090,1535,3551771,431277,...,305.0,15598.0,43241.0,Nissan,Pathfinder,2014,Fraud Reported,69774.695312,0.0,Fraud Reported


# Incident Date correction

In [113]:
df_backup_1 = df_backup.copy()

In [114]:
df_backup['INCIDENT_DATE'] = df_backup.INCIDENT_DATE + pd.DateOffset(months=7)

In [115]:
max(df_backup['INCIDENT_DATE'])

Timestamp('2024-04-30 00:00:00')

In [118]:
df_backup['INCIDENT_DATE'] = df_backup['INCIDENT_DATE'].astype('str')

In [119]:
DF_BACKUP=session.createDataFrame(
        df_backup.values.tolist(),
        schema=df_backup.columns.tolist())
DF_BACKUP.write.mode("overwrite").save_as_table("FDC_Insurance.PUBLIC.AUTO_INSURANCE_CLAIMS_HARMONIZER_INSIGHT")

# Run below query once Snowflake table is created

### ALTER TABLE FDC_INSURANCE.PUBLIC.AUTO_INSURANCE_CLAIMS_HARMONIZER_INSIGHT ADD COLUMN INCIDENT_DATETIME DATE

### UPDATE FDC_INSURANCE.PUBLIC.AUTO_INSURANCE_CLAIMS_HARMONIZER_INSIGHT SET INCIDENT_DATETIME = TO_DATE (INCIDENT_DATE)